In [1]:
import os

import re
import time
import pandas as pd
import numpy as np

from dotenv import load_dotenv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver import ActionChains

load_dotenv()


# ElementClickInterceptedException - неможна клацнуть по елементу
# NoSuchElementException - такого елементу неіснує

True

In [165]:
# funcs

def login(driver):
    email = os.getenv("EMAIL")
    password = os.getenv("PASS")
    
    
    login_button = find_elems(driver, {
                                        "by_attr": By.ID, 
                                        "attr_data": "user-menu",
                                        "attr_target": "div"
                                        }
                                )[0]
    login_button.click()
    
    
    block_login_by = find_elems(driver, {
                                        "by_attr": By.CLASS_NAME, 
                                        "attr_data": "lsidDialog__section",
                                        }
                               )
    login_by_email = find_elems(block_login_by[0], {
                                        "by_attr": By.TAG_NAME, 
                                        "attr_data": "button",
                                        }
                               )[-1]
    login_by_email.click()
    
    block_inputs = find_elems(driver, {
                                        "by_attr": By.CLASS_NAME, 
                                        "attr_data": "lsidDialog__section",
                                        }
                               )
    input_fields = find_elems(block_inputs[0], {
                                        "by_attr": By.TAG_NAME, 
                                        "attr_data": "input",
                                        }
                             )
    time.sleep(1)
    input_fields[0].send_keys(email)
    input_fields[1].send_keys(password)
    input_fields[1].send_keys(Keys.RETURN)  
      
    time.sleep(5)  # для закриття вспливаючого вікна
    return driver


def get_driver():
    path_to_driver = "./chromedriver"
    chrome_service = Service(path_to_driver)
    options = Options()
#     options.add_argument("headless")
    return webdriver.Chrome(
        service=chrome_service,
        options=options
    )


def close_cookies(driver):
    """
    закриття вікна з підтвердження куків
    """
    
    elems = find_elems(driver, {
                                "by_attr": By.ID, 
                                "attr_data": "onetrust-accept-btn-handler",
                                }
                      )
    if not elems:
        return driver
    elems[0].click()
    return driver
    
    
def find_elems(target, args):
    args_attr_data = args["attr_data"]
    if args["by_attr"] == By.XPATH and args["by_text"] is False:
        attr_data =  f"//{'attr_target'}[starts-with(@class, '{args_attr_data}')]" 
    elif args["by_attr"] == By.XPATH and args["by_text"] is True:
        attr_data =  f"//{'attr_target'}[text()='{args_attr_data}']"
    else:
        attr_data = args["attr_data"]
    
    count = 0
    while count <= 5:
        try:
            elements = target.find_elements(args["by_attr"], attr_data)
            if len(elements) == 0:
                count += 1
                continue
            break
        except NoSuchElementException:
            time.sleep(1)
            count += 1
    
    return elements


def get_my_champs(driver):
    """
    створення списку кортежів, назв чемпів + частин посилання, на всі чемпи в моїх лігах
    """
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    champs = []
    for champ in soup.find_all("div", class_="leftMenu__item"):
        href = champ.find('a', class_="leftMenu__href")["href"]
        champs.append((champ["title"], href))
    return champs


def get_champ_dict_from_file():
    champ_dict = {}
    with open("champ_list.txt", "r") as f:
        for line in f:
            name, href = line.split(",")
            champ_dict[name.strip()] = href.strip()
    return champ_dict


def handle_one_champ(driver, champ):
    url_future_matches = f"https://www.soccerstand.com{champ}fixtures/"
    url_past_matches = f"https://www.soccerstand.com{champ}results/"
    driver.get(url_past_matches)  
    time.sleep(3)
    # допрацювати
    

def get_future_matches(driver, days=0):
    # переключення дня 
    for _ in range(days):
        pass
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    a = soup.find_all("div", class_=re.compile("event__header top"))
    
    return driver.page_source, a

In [177]:
try:
    url = "https://www.soccerstand.com/ru/"
    driver = get_driver()
    driver.get(url)
    driver = close_cookies(driver)
    
    driver = login(driver)  

    # отримання списку(словаря) моїх чемпіонатів
#     champs = get_my_champs(driver)
    champ_dict = get_champ_dict_from_file()
    
#     for champ in champ_list:
#         handle_one_champ(driver, champ)
    
    data, a = get_future_matches(driver, days=0)
    time.sleep(3)
    raise
    
        
    
finally:
    driver.quit()


RuntimeError: No active exception to reraise

In [187]:
for champ_block in a: 
    champ_country_name = champ_block.find("span", class_="event__title--type").text # назва чемпа країни
    champ_name = champ_block.find("span", class_="event__title--name").text # назва чемпа
    full_champ_name = f"{champ_country_name}: {champ_name}"
    
    if full_champ_name not in champ_dict.keys():
        print(full_champ_name)
#     raise

БОЛГАРИЯ: Первая лига - Conference League Group
БРАЗИЛИЯ: Чемпионат Бразилии C - Первый этап
ГЕРМАНИЯ: Вторая Бундеслига - Понижение (плей-офф)
ИНДОНЕЗИЯ: Лига 1 - Лига чемпионов - плей-офф
НИДЕРЛАНДЫ: Высшая лига - Борьба за выживание - плей-офф
ПОЛЬША: Первый дивизион - Повышение - плей-офф
ТУНИС: Высшая лига - Чемпионшип - плей-офф
УРУГВАЙ: Премьер-лига - Торнео Интермедио
ШВЕЙЦАРИЯ: Суперлига - Понижение (плей-офф)
ЭКВАДОР: Лига Про - Первый этап


In [120]:
# отримання всіх матчів для контретного чемпа 
for x in a:
    print(x.text)
    b = x.find_next_sibling("div")
    while "id" in b.attrs.keys():
        print(f"\t{b.text}")
        b = b.find_next_sibling("div")
    print()

АЛЖИРПервый дивизионТаблица
	18:30АС Олимпик де ШлефМагра--
	18:30КабилиМК Алжир--
	18:30КонстантинАрба--
	18:30ПарадуОран--
	18:30ХенчелаСаура--
	18:30ШелгумЕС Сетиф--
	18:30Эль-БаядБелуиздад--

АРГЕНТИНАПримера НасьональТаблица
	ЗавершенРасинг КордоваКильмес01(0)(1)

АРМЕНИЯПремьер-лигаТаблица
	16:00АлашкертПюник--
	18:00УрартуАрарат--

БОЛГАРИЯПервая лига - Conference League GroupТаблица
	19:30Ботев ПловдивАрда--
	19:30Славия СофияЛокомотив София--

БРАЗИЛИЯЧемпионат БразилииТаблица
	ЗавершенВаско да ГамаФламенго14(0)(4)

БРАЗИЛИЯЧемпионат Бразилии C - Первый этапТаблица
	ЗавершенСан-БернардоФигейренсе10(0)(0)

ВЬЕТНАМЛига VТаблица Live
	ЗавершенХан ХоаВьеттел00(0)(0)
	83 Сонглам НгеанХошимин21(2)(1)

ГЕРМАНИЯВторая Бундеслига - Понижение (плей-офф)Сетка
	21:45АрминияВехен--

ГРУЗИЯЛига ЭровнулиТаблица
	16:00ШукураСамгурали--
	18:00ТелавиГагра--

ЕГИПЕТПремьер-лигаТаблица
	19:00ФаркоПирамидс--
	19:00Эль-ДахлиеХарас Эль-Ходуд--
	21:30ФьючерЭнппи--

ИНДОНЕЗИЯЛига 1 - Лига чемпионов - 